In [1]:
import numpy as np

## Ejercicio 1

### Algoritmo Backpropagation

Consideremos el caso de redes *feed-forward multilayer* en donde además de la capa de entrada tenemos N-1 capas ocultas. Una dada capa $l$ recibe como entrada la salida de la capa $l-1$, para cada uno de los $\mu$ ejemplos de entrenamiento. Es decir, la salida de una capa está dada por

$$h_j^{\mu}(l, l+1) = \sum_k \omega_{j, k}^{l, l+1} g\left(h_j^{\mu} (l, l+1)\right)$$

donde $\omega_{j, k}^{l, l+1}$ es la matriz de pesos que conecta la capa $l$ con la capa $l+1$, $g$ es la función de activación aplicada a la salida de la capa $l-1$. La función de activación puede ser una función sigmoide, tangente hiperbólica (tanh), ReLU, entre otras. La elección de la función de activación depende del problema específico y de la arquitectura de la red neuronal.

Por lo tanto, denotamos la entrada a la capa siguiente esta dada por:

$$ V_j^{\mu} (l, l+1) = g(h_j^{\mu}(l, l+1))$$

Notar que en la primera capa, se tiene que 

$$h_j^{\mu}(0, 1) = \sum_k \omega_{j, k}^{0,1} x_k^\mu$$

donde $\mathbf{x}^\mu$ son los vectores de entrada.

Por lo tanto, la salida de la red se puede escribir como:

$$o_j^\mu = V_j^\mu (N, N-1) = g\left(\sum_{k_1} \omega_{j, k_1}^{N, N-1} g\left(\sum_{k_2} \omega_{k_1, k_2}^{N-1, N-2} g\left(...g\left(\sum_{k_N} \omega_{k_{N-1}, k_{N}}^{0, 1} x_{k_N}^\mu \right) ... \right)\right)\right)$$

En resumen, la salida de la red depende de los pesos que se le asignan a las matrices $\mathbf{\omega}^{l, l+1}$. Por lo tanto, el objetivo del entrenamiento es encontrar los pesos de esas matrices para que la salida de la red $\mathbf{o}^\mu$ sea igual al target $\mathbf{y}^\mu$.

En el algoritmo de backpropagation, se busca realizar un descenso por gradiente de alguna función error $E[\mathbf{\omega}]$ de la salida de la red comparada con el target, comenzando desde la ultima capa hasta la primera. 

La función error más usual es el error cuadrático medio, es decir:

$$ E[\omega] = \frac{1}{2} \sum_{\mu, i} \left[y_i^\mu - o_i^\mu(\mathbf{\omega}) \right]^2 $$

A modo de ejemplo, consideremos el caso de una red de 2 capas, es decir la capa de entrada más una capa oculta. Por lo tanto, la función error resulta:

$$ E[\omega] = \frac{1}{2} \sum_{\mu, i} \left[y_i^\mu - g\left(\sum_j \omega_{i, j}^{1, 2} g\left(\sum_k \omega_{j, k}^{0,1} x_k^\mu\right) \right) \right]^2 $$

 para implementar el algoritmo de *backprojection*, consideramos primero el caso de la capa oculta a la salida, y luego de la entrada a la capa oculta.

### HIDDEN $\rightarrow$ OUTPUT

$$\Delta \omega_{i,j}(1, 2) = \eta \frac{\partial E}{\partial \omega_{i, j}(1, 2)} = \frac{\partial E}{\partial o_i} \frac{\partial o_i}{\partial h_i} \frac{\partial h_i}{\partial \omega_{i, j}}$$

- $\frac{\partial E}{\partial o_i} = \eta \sum_\mu y_i^\mu - o_i^\mu$: representa el error entre la salida y el target.
- $\frac{\partial o_i}{\partial h_i} = g'(h_i^\mu)$: es la derivada de la función de activación de todos los ejemplos.
- $\frac{\partial h_i}{\partial \omega_{i, j}} = \left(\sum_k \omega_{j, k} x_k^\mu\right) = V_j^\mu$: es la suma de los ejemplos de la salida de la capa anterior.

Por lo tanto, resulta:

$$\Delta \omega_{i,j}(1, 2)= \eta \sum_\mu \left(y_i^\mu - o_i^\mu \right) g'(h_i^\mu) V_j^\mu $$

en donde definimos $\delta_i^\mu = g'(h_i^\mu) \left(y_i^\mu - o_i^\mu \right)$, por lo tanto 

$$\boxed{\Delta \omega_{i,j}(1, 2) = \eta \sum_\mu \delta_i^\mu V_j^\mu}$$

### INPUT $\rightarrow$ HIDDEN

$$\Delta \omega_{j,k}(0, 1) = \eta \frac{\partial E}{\partial \omega_{j, k}(0, 1)} = \eta \frac{\partial E}{\partial V_j^\mu} \frac{\partial V_j^\mu}{\partial \omega_{j,k}(0, 1)}$$

- $\frac{\partial E}{\partial V_j^\mu} = \sum_{\mu, i} (y_i^\mu - o_i^\mu) g'(h_i^\mu) \omega_{i,j}(1, 2)$
- $\frac{\partial V_j^\mu}{\partial \omega_{j,k}(0, 1)} =  g'(h_j^\mu)x_k^\mu$

por ende, resulta:
$$\Delta \omega_{j,k}(0, 1) = \eta \sum_{\mu, i} (y_i^\mu - o_i^\mu) g'(h_i^\mu) \omega_{i,j}(1, 2)g'(h_j^\mu)x_k^\mu = \eta \sum_{\mu, i} \delta_i^\mu  \omega_{i,j}(1, 2)g'(h_j^\mu)x_k^\mu$$

en donde definimos $\delta_j^\mu = g'(h_j^\mu) \sum_i \delta_i^\mu  \omega_{i,j}(1, 2)$, por lo tanto

$$\boxed{\Delta \omega_{j,k}(0, 1) = \eta \sum_\mu \delta_j^\mu x_k^\mu} $$


Funciones de activacion

In [2]:
activation = lambda h: np.tanh(h)
derivate_act = lambda h: 1 - np.tanh(h)**2

## Arquitectura 1

### Sin bias

In [3]:
# entradas de la compuerta
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
# salidas
Y = np.array([[0], [1], [1], [0]])

# Semilla aleatoria para reproducibilidad
np.random.seed(42)

# Pesos entre capa de entrada y capa oculta (2x2)
weights_input_hidden = np.random.rand(2, 2) 
# Pesos entre capa oculta y capa de salida (2x1)
weights_hidden_output = np.random.rand(1, 2)

# Definimos la tasa de aprendizaje
learning_rate = 0.1

In [4]:
def NN(x, w_ih, w_ho):
    Vj = np.zeros((X.shape[0], w_ih.shape[0]))
    oi = np.zeros((Vj.shape[0], w_ho.shape[0]))

    for mu in range(X.shape[0]):    # itero en todos los ejemplos
        Vj[mu] = activation(np.einsum('jk, k -> j', w_ih, x[mu]))   # es lo mismo q producto escalar
        oi[mu] = activation(np.einsum('ij, j -> i', w_ho, Vj[mu]))

        # print(Vj[mu].shape, oi[mu].shape)
    
    return oi

In [5]:
epochs = 1000000

# print(NN(X, weights_input_hidden, weights_hidden_output))

# Aprendizaje: Algoritmo de retroproyeccion
print(X.shape, Y.shape)
for i in range(epochs):
    grad_w_0_1, grad_w_1_2 = 0, 0   # pongo los gradientes en 0

    for mu, x in enumerate(X):  #itero en los ejemplos
        # weights_input_hidden shape (2, 2)
        # weights_hidden_output shape: (1, 2)
        # x shape (2, )

        x = x.reshape((2,1))
        # Forward propagation
        hj = np.dot(weights_input_hidden, x)
        hi = np.dot(weights_hidden_output, activation(hj))

        # backpropagation
        # hidden -> output
        delta_i_mu = derivate_act(hi)*(Y[mu] - activation(hi))
        grad_w_1_2 += np.dot(delta_i_mu, activation(hj).T)

        # input -> hidden
        delta_j_mu = derivate_act(hj) * np.dot(weights_hidden_output.T, delta_i_mu)
        grad_w_0_1 += np.dot(delta_j_mu, x.T)
        
    # actualizo los pesos
    weights_input_hidden += learning_rate * grad_w_0_1
    weights_hidden_output += learning_rate * grad_w_1_2

(4, 2) (4, 1)


In [6]:
# TEST
print(NN(X, weights_input_hidden, weights_hidden_output))

[[ 0.        ]
 [ 0.99573709]
 [ 0.99573711]
 [-0.00685007]]


### Con bias

En el caso de capa con bias, se suma una constante a la salida de la neurona que se corresponde con el bias propio de la neurona, es decir:

$$ h_j^{\mu}(l, l+1) = \sum_k \omega_{j, k}^{l,l+1} x_k^\mu + b_l$$

La lógica de las conexiones es la misma que la explicada para el caso sin bias, es decir que la salida de una capa se conecta a la entrada de la siguiente y así sucesivamente hasta la salida. Por lo tanto, la salida de la red resulta:

$$o_j^\mu = V_j^\mu (N, N-1) = g\left(\sum_{k_1} \omega_{j, k_1}^{N, N-1} g\left(\sum_{k_2} \omega_{k_1, k_2}^{N-1, N-2} g\left(...g\left(\sum_{k_N} \omega_{k_{N-1}, k_{N}}^{0, 1} x_{k_N}^\mu +b_0\right) ... \right)+b_{N-1}\right)+b_N\right)$$


por lo tanto, en el algoritmo de backpropagation debemos agregar la corrección del bias. Para ello utilizamos el mismo método de descenso por gradiente, comenzando desde la última capa hasta la primera.

Consideremos nuevamente el caso de 2 capas como el del ejemplo anterior, en donde seguimos notación del Hertz

$$h_j^\mu = \sum_k \omega_{j, k}^{0,1} x_k^\mu + b_0 $$
$$h_i^\mu = \sum_j \omega_{i, j}^{1, 2} g(h_j^\mu) + b_1$$
$$V_j^\mu =  g\left(\sum_k \omega_{j, k}^{0,1} x_k^\mu + b_0 \right)$$
$$o_i^\mu = g(h_i^\mu)$$



La función error resulta

$$ E[\omega] = \frac{1}{2} \sum_{\mu, i} \left[y_i^\mu - g\left(\sum_j \omega_{i, j}^{1, 2} g\left(\sum_k \omega_{j, k}^{0,1} x_k^\mu + b_0 \right) + b_1\right) \right]^2 $$

#### HIDDEN $\rightarrow$ OUTPUT

$$\frac{\partial E[\mathbf{\omega}]}{\partial b_1} = \frac{\partial E}{\partial o_i} \frac{\partial o_i}{\partial h_i} \frac{\partial h_i}{\partial b_1} $$

- $\frac{\partial E}{\partial o_i} = \sum_\mu y_i^\mu - o_i^\mu$: representa el error entre la salida y el target.
- $\frac{\partial o_i}{\partial h_i} = g'(h_i^\mu)$: es la derivada de la función de activación de todos los ejemplos.
- $\frac{\partial h_i}{\partial b_1} = 1$

por lo tanto, se tiene que $\frac{\partial E[\mathbf{\omega}]}{\partial b_1} = \sum_\mu (y_i^\mu - o_i^\mu)g'(h_i^\mu) = \delta_i^\mu $ entonces

$$\boxed{\Delta b_1 = \eta \sum_\mu \delta_i^\mu}$$

#### INPUT $\rightarrow$ HIDDEN

$$\frac{\partial E[\mathbf{\omega}]}{\partial b_0} = \frac{\partial E}{\partial V_j^\mu} \frac{\partial V_j^\mu}{\partial b_0}$$ 


- $\frac{\partial E}{\partial V_j^\mu} = \sum_{\mu, i} (y_i^\mu - o_i^\mu) g'(h_i^\mu) \omega_{i,j}(1, 2)$
- $\frac{\partial V_j^\mu}{\partial b_0} =  g'(h_j^\mu)$

por lo tanto, se tiene que $\frac{\partial E[\mathbf{\omega}]}{\partial b_0} = \sum_{\mu, i} (y_i^\mu - o_i^\mu) g'(h_i^\mu) \omega_{i,j}(1, 2) g'(h_j^\mu) = \sum_{\mu} g'(h_j^\mu) \sum_i \delta_i^\mu \omega_{i, j}(1, 2) = \sum_\mu \delta_j^\mu$

como resultado, se obtiene que

$$ \boxed{\Delta b_0 = \eta \sum_\mu \delta_j^\mu} $$


In [7]:
# entradas de la compuerta
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
# salidas
Y = np.array([[0], [1], [1], [0]])

# Semilla aleatoria para reproducibilidad
np.random.seed(42)

# Pesos entre capa de entrada y capa oculta (2x2)
weights_input_hidden = np.random.rand(2, 2) 
# Pesos entre capa oculta y capa de salida (2x1)
weights_hidden_output = np.random.rand(1, 2)
#Bias
b0 = np.random.rand(2)
b1 = np.random.rand(1)

# Definimos la tasa de aprendizaje
learning_rate = 0.2

In [8]:
def NN_bias(x, w_ih, w_ho, b1, b0):
    Vj = np.zeros((X.shape[0], w_ih.shape[0]))
    oi = np.zeros((Vj.shape[0], w_ho.shape[0]))

    for mu in range(X.shape[0]):    # itero en todos los ejemplos
        
        Vj[mu] = activation(np.einsum('jk, k -> j', w_ih, x[mu])+b0)
        oi[mu] = activation(np.einsum('ij, j -> i', w_ho, Vj[mu])+b1)
        # print(Vj[mu].shape, oi[mu].shape, b0.shape, b1.shape)
    
    return oi

In [9]:


# print(NN(X, weights_input_hidden, weights_hidden_output))

# Aprendizaje: Algoritmo de retroproyeccion
print(X.shape, Y.shape)


# error de la red
error = np.ones_like(Y)
epsilon = 1e-02
epochs = 0

# si quier definir un numero fijo de epochs
# for i in range(epochs):
#     grad_w_0_1 = np.zeros_like(weights_input_hidden)
#     grad_w_1_2 = np.zeros_like(weights_hidden_output)
#     grad_b0 = np.zeros_like(b0)
#     grad_b1 = np.zeros_like(b1)

#     for mu, x in enumerate(X):  # itero en los ejemplos
#         x = x.reshape((2, 1))
#         # Forward propagation
#         hj = np.dot(weights_input_hidden, x) + b0.reshape((2, 1))
#         Vj = activation(hj)
#         hi = np.dot(weights_hidden_output, Vj) + b1
#         oi = activation(hi)

#         # Backpropagation
#         # hidden -> output
#         delta_i_mu = derivate_act(hi) * (Y[mu] - oi)
#         grad_w_1_2 += np.dot(delta_i_mu, Vj.T)
#         grad_b1 += delta_i_mu.flatten()

#         # input -> hidden
#         delta_j_mu = derivate_act(hj) * np.dot(weights_hidden_output.T, delta_i_mu)
#         grad_w_0_1 += np.dot(delta_j_mu, x.T)
#         grad_b0 += delta_j_mu.flatten()

#     # Actualizo los pesos y los bias
#     weights_input_hidden += learning_rate * grad_w_0_1
#     weights_hidden_output += learning_rate * grad_w_1_2
#     b0 += learning_rate * grad_b0
#     b1 += learning_rate * grad_b1

while np.sqrt(np.sum(error**2)) > epsilon:
    epochs += 1
    grad_w_0_1 = np.zeros_like(weights_input_hidden)
    grad_w_1_2 = np.zeros_like(weights_hidden_output)
    grad_b0 = np.zeros_like(b0)
    grad_b1 = np.zeros_like(b1)

    for mu, x in enumerate(X):  # itero en los ejemplos
        x = x.reshape((2, 1))
        # Forward propagation
        hj = np.dot(weights_input_hidden, x) + b0.reshape((2, 1))
        Vj = activation(hj)
        hi = np.dot(weights_hidden_output, Vj) + b1
        oi = activation(hi)

        # Backpropagation
        # hidden -> output
        delta_i_mu = derivate_act(hi) * (Y[mu] - oi)
        grad_w_1_2 += np.dot(delta_i_mu, Vj.T)
        grad_b1 += delta_i_mu.flatten()

        # input -> hidden
        delta_j_mu = derivate_act(hj) * np.dot(weights_hidden_output.T, delta_i_mu)
        grad_w_0_1 += np.dot(delta_j_mu, x.T)
        grad_b0 += delta_j_mu.flatten()

    # Actualizo los pesos y los bias
    weights_input_hidden += learning_rate * grad_w_0_1
    weights_hidden_output += learning_rate * grad_w_1_2
    b0 += learning_rate * grad_b0
    b1 += learning_rate * grad_b1

    error = Y - NN_bias(X, weights_input_hidden, weights_hidden_output, b1, b0)

    print(np.sum(error**2))
# error = Y - NN_bias(X, weights_input_hidden, weights_hidden_output, b1, b0)

print(f'Error: {np.sqrt(np.sum(error)**2)}')
print()

(4, 2) (4, 1)
1.002312720670148
0.9789794114402482
0.975383517323086
0.9732428132581579
0.9710843412418475
0.9688688961618783
0.9665885929014346
0.9642363069684003
0.9618053268337385
0.9592893892421348
0.956682724793521
0.9539801091965106
0.9511769193177879
0.9482691928567775
0.9452536901805014
0.9421279565917535
0.9388903830847481
0.9355402634791867
0.9320778457277509
0.9285043751717044
0.9248221275792391
0.9210344299417597
0.9171456672211423
0.9131612735294747
0.9090877065732497
0.9049324045967725
0.9007037255049983
0.8964108683250189
0.8920637776692057
0.8876730323825919
0.8832497200817803
0.8788052998089435
0.8743514555145062
0.869899943523323
0.8654624375048414
0.8610503747286071
0.8566748075137769
0.8523462637505399
0.8480746201663112
0.843868991625978
0.8397376392033915
0.8356878990658307
0.8317261334130291
0.8278577038567753
0.8240869667707306
0.8204172893376894
0.8168510843220738
0.813389861037735
0.8100342895903283
0.8067842752597245
0.8036390398468515
0.8005972069240246
0.79

In [10]:
# TEST
print(NN_bias(X, weights_input_hidden, weights_hidden_output, b1, b0))

[[8.54266033e-05]
 [9.94436985e-01]
 [9.94441737e-01]
 [6.17312193e-03]]
